In [19]:
import os
from src.vectorstore import VectorstoreHandler
from src.models import init_emb, init_llm
from src.rag_tools import build_rag_chain
import textwrap


# Config JSON for RAG specifications
RAG_CONFIG = {
    "book": {
        "subfolder": "book",
        "llm_name": "Llama3.2-3b",
        "emb_name": "hf-minilm-l6-v2",
        "k": 10,
    },
    "lectures": {
        "subfolder": "lectures",
        "llm_name": "Llama3.2-3b",
        "emb_name": "hf-minilm-l6-v2",
        "k": 10,
    },
}

# Global directory for sources
SOURCES_DIR = "sources"


# Function to process RAG based on config
def create_rag(config, handler):
    """
    Create a RAG chain based on the provided configuration.

    Args:
        config (dict): Configuration for the RAG (subfolder, LLM, embedding, k, etc.).
        handler (VectorstoreHandler): Handler for managing vectorstores.

    Returns:
        tuple: A tuple containing the RAG chain and its retriever.
    """
    subfolder = config["subfolder"]
    llm_name = config["llm_name"]
    emb_name = config["emb_name"]
    k = config["k"]

    # Paths and model initialization
    dir_path = os.path.join(SOURCES_DIR, subfolder)
    emb = init_emb(emb_name)
    llm = init_llm(llm_name)

    # Build vectorstore and retriever
    vs = handler.build_vectorstore(dir_path, emb, emb_name)
    retriever = handler._init_retriever(vs, dir_path, k)

    # Build and return the RAG chain
    chain = build_rag_chain(retriever, llm)
    return chain, retriever

    import textwrap

def generate_answers(prompt, rags):

    rag_responses = {}
    print(f"\nQuerying all RAGs for prompt: '{prompt}'\n{'=' * 80}")
    for rag_name, (chain, retriever) in rags.items():
        print(f"--- Querying RAG: {rag_name} ---")
        output = chain.invoke(prompt)
        # Store the raw response and associated documents
        rag_responses[rag_name] = {
            "answer": output["answer"],
            "docs": output["docs"],
        }
    return rag_responses

def generate_and_display_aggregate_response(aggregation_llm, agent_answers, prompt):
    """
    Generates an aggregated response from the answers of individual agents and prints it.
    
    Args:
        aggregation_llm: The LLM used for aggregating responses.
        agent_answers (dict): A dictionary with agent names as keys and their answers as values.
        prompt (str): The prompt provided to all agents.

    Returns:
        None
    """
    # Step 1: Construct the aggregation prompt
    aggregation_prompt = f"Here are the answers from different sources for the prompt: '{prompt}':\n"
    for agent_name, answer in agent_answers.items():
        aggregation_prompt += f"\n[{agent_name}] {answer}\n"
    aggregation_prompt += (
        "\nPlease provide a concise, technical, and structured summary of the above, "
        "formatted for terminal display with a wrapping width of 80 characters."
        "If the answer contains mathematical notation, default to using LaTeX-syntax."
    )

    # Step 2: Generate the aggregated response
    aggregated_output = aggregation_llm.invoke(aggregation_prompt)
    if isinstance(aggregated_output, str):
        aggregated_answer = aggregated_output
    else:
        aggregated_answer = aggregated_output.content['answer']

    # Step 3: Print the aggregated answer first
    print("\nAggregated Answer:\n" + "=" * 80)
    wrapped_aggregated_answer = textwrap.fill(aggregated_answer, width=80)
    print(wrapped_aggregated_answer + "\n")

    # Step 4: Print detailed answers from each agent
    print("\nDetailed Responses from Agents:\n" + "=" * 80)
    for agent_name, answer in agent_answers.items():
        if isinstance(answer, str):
            answer = answer
        else:
            answer = answer['answer']

        wrapped_answer = textwrap.fill(f"[{agent_name}] {answer}", width=80)
        print(wrapped_answer + "\n")

In [20]:
# Initialize the vectorstore handler and construct the rags
handler = VectorstoreHandler(SOURCES_DIR, force_rebuild=False)

rags = {}
for rag_name, rag_config in RAG_CONFIG.items():
    print(f"Setting up RAG: {rag_name}")
    rags[rag_name] = create_rag(rag_config, handler)

Setting up RAG: book
Loading existing vectorstore for sources/book...
Setting up RAG: lectures
Loading existing vectorstore for sources/lectures...


In [21]:
AGGREGATION_LLM_NAME = "Llama3.2-3b"  # Change to your preferred LLM
aggregation_llm = init_llm(AGGREGATION_LLM_NAME)

# Example usage: querying all RAGs
test_prompt = "What are the assumptions used for krieging?"

rag_responses = generate_answers(test_prompt,rags)
generate_and_display_aggregate_response(aggregation_llm,rag_responses, test_prompt)


Querying all RAGs for prompt: 'What are the assumptions used for krieging?'
--- Querying RAG: book ---
--- Querying RAG: lectures ---

Aggregated Answer:
**Kriging**  **Definition**: Kriging is a method of interpolation used to
estimate the value of an unobserved point in a spatial field, given observed
data.  **Mathematical Formulation**:  Given jointly Gaussian column vectors X
and X , with means (µ, µ) and covariances (Σ, Σ), we have:  X | X ∼ N(µ + Σ −1(X
- µ), Σ − Σ −1Σ)  The kriging predictor is given by:  E[X | X] = µ + Σ −1(X - µ)
**Properties**:  * For a Gaussian random field, the point prediction E[X | X] is
called the kriging predictor. * The kriging estimator satisfies:  E[X(A)] = λ|A|
when Y is stationary and isotropic.  **Applications**:  * Spatial analysis *
Geostatistics * Remote sensing  **Code Structure**: ```latex
\documentclass{article} \usepackage{amsmath}  \begin{document}
\section*{Kriging}   Kriging is a method of interpolation used to estimate the
value of an 

In [ ]:
aggregated_output

In [8]:
print(aggregated_output)

Here is a consolidated summary of the assumptions for Kriging:

**Common Assumptions:**

1. **Randomness**: The data points must be randomly and uniformly distributed within the area of interest.
2. **Stationarity**: The underlying process (e.g., spatial distribution of a field) is a stationary random process, meaning that the mean and variance do not change with location, and autocorrelation depends only on distance between points.

**Additional Assumptions:**

1. **Zero Mean Gaussian Process**: The random field X must be a zero-mean Gaussian process.
2. **Gaussian Observations**: The observations Y are from this Gaussian process.
3. **Estimated Covariance Parameters**: The covariance parameters Θ, including σ^2, are estimated before computing predictions.
4. **Stationarity and Isotropy (if applicable)**: Stationarity and isotropy of the random field Y may be assumed, depending on the context.

**Note:** While Kriging can be used under various assumptions and conditions, these fundame